In [0]:
# debugging and formatter
from IPython.core.debugger import set_trace

import pandas as pd
import numpy as np
import os
import random
import matplotlib.pyplot as plt
import pickle
import time

%matplotlib inline

# Anonymizing Data
While Maintaining Semantic Relationships and Distributions

In [4]:
!pip install faker
import csv
from collections import defaultdict
from faker import Faker

fake = Faker()

In [0]:
df = pd.read_csv(
    "fake_data.csv",
    header=None,
    names=["name", "email", "country", "gender", "avg_rating", "long_tailed", "target"],
)
df.head()

FileNotFoundError: ignored

Function that will anonymize the data.
Create mappings of fields that need to be anonymized.

In [0]:
def anonymize_rows(rows):
    """
    Rows is a dict iterable that contains the fields that need to be anonymized.
    """
    names = defaultdict(fake.last_name)
    emails = defaultdict(fake.free_email)

    # Iterate over the rows from the file and yield anonymized rows.
    for row in rows:
        # Replace name and email fields with faked fields.
        row["name"] = names[row["name"]]
        row["email"] = emails[row["email"]]

        # Yield the row back to the caller
        yield row

Function that takes our source file and outputs our target file.

In [0]:
def anonymize(source, target):
    """
    The source argument is a path to a csv file containing data to anonymize,
    while target is a path to write the anonymized csv data to.
    """
    with open(source, "r", newline="") as f:
        with open(target, "w") as o:
            # Use DictReader to easily extract fields
            reader = csv.DictReader(f)
            next(reader)
            writer = csv.DictWriter(o, reader.fieldnames)

            # Read and anonymize data, writing to target file.
            for row in anonymize_rows(reader):
                writer.writerow(row)

In [0]:
anonymize("fake_data.csv", "anonymized_fake_data.csv")

In [0]:
df2 = pd.read_csv(
    "anonymized_fake_data.csv",
    header=None,
    names=["name", "email", "country", "gender", "avg_rating", "income", "target"],
)

In [0]:
df2.head(5)

We can see that the distribution remains the same.


In [0]:
df_names = df["name"].value_counts()[:10]
df2_names = df2["name"].value_counts()[:10]

In [0]:
f = plt.figure(1, figsize=(16, 8))
ax1 = f.add_subplot(2, 2, 1)
ax1.title.set_text("DF")
_ = ax1.bar(df_names.index, df_names.values)

ax2 = f.add_subplot(2, 2, 2)
ax2.title.set_text("DF2")
_ = ax2.bar(df2_names.index, df2_names.values)